In [1]:
import os
import re

from dotenv import load_dotenv

In [2]:
from langchain_community.vectorstores import FAISS
from utils import load_embeddings, initialize_llm
from prompt import contextualize_q_system_prompt, system_prompt


from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
load_dotenv()

True

In [4]:
os.environ['GOOGLE_API_KEY'] = os.environ.get('GOOGLE_API_KEY')
os.environ['GROQ_API_KEY'] = os.environ.get('GROQ_API_KEY')

In [5]:
embeddings = load_embeddings(embd_type='GoogleGenAI')

## Load the persisted database from disk

In [6]:
# load the VectorDB
db_directory_path="Gale_Encyclopedia_of_Med_db"
docsearch = FAISS.load_local(db_directory_path, embeddings, allow_dangerous_deserialization=True)

## Make a Retriever (Similarity Search)

In [7]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5}) 

In [8]:
llm = initialize_llm(temp=0.6)

## Make a chain

In [9]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [10]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [11]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [12]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [13]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

## Get a response

In [14]:
def get_response(query):
    ai_msg = conversational_rag_chain.invoke(
        {"input": query},
        config={
            "configurable": {"session_id": "abc123"}
        },  # constructs a key "abc123" in `store`.
    )
    return ai_msg

In [15]:
def process_DSR1_response(query):
    res = get_response(query)
    return res, re.sub(r"<think>.*?</think>\n\n?", "", res["answer"], flags=re.DOTALL)

## Questions and its responses

In [16]:
res, response = process_DSR1_response("Tell me about the kidney disease?")

print(response)

Kidney disease, also known as renal disease, refers to conditions that reduce kidney function. The kidneys, bean-shaped organs located below the rib cage, filter blood to remove waste and regulate water and electrolyte levels. They also produce renin for blood pressure control and calcitriol for bone health. Types of kidney disease include congenital diseases like Polycystic Kidney Disease (PKD), which causes fluid-filled cysts and affects about half a million people in the U.S., and acute kidney diseases that develop suddenly. End-Stage Renal Disease (ESRD) occurs when kidney function drops to 10-15%, often requiring dialysis or transplant. Causes include diabetes, high blood pressure, genetics, and infections, with symptoms like blood in urine, proteinuria, and edema. Prevention involves hydration, hygiene, and avoiding certain medications. Treatment may include medications, diet changes, dialysis, or transplant, though PKD has no cure. Prognosis varies, with infants often not surviv

In [17]:
res, response = process_DSR1_response("What are its Causes & symptoms?")

print(response)

**Causes of Kidney Disease:**

1. **Diabetes:** The leading cause of chronic kidney disease, accounting for 33-40% of new cases. High blood sugar damages the nephrons, reducing their ability to filter blood effectively.
2. **High Blood Pressure (Hypertension):** The second leading cause, responsible for 27-30% of new cases. It damages the capillaries in the nephrons, impairing blood filtration.
3. **Polycystic Kidney Disease (PKD):** An inherited disorder causing cysts to form on the nephrons, disrupting kidney function.
4. **Kidney Cancer:** Though uncommon, accounting for 2% of cancer cases, it can lead to kidney failure.
5. **Obstructions:** Such as kidney stones, which can block urine flow and damage the kidneys.
6. **Infections:** Including pyelonephritis and reflux nephropathy, which can cause inflammation and scarring.
7. **Systemic Lupus Erythematosus (SLE):** An autoimmune disease that can damage kidney tissues.
8. **Amyloidosis:** A condition where abnormal proteins deposit i

In [18]:
res, response = process_DSR1_response("How can it be cured?")

print(response)

The treatment of kidney disease depends on the type, severity, and underlying cause. Here are some common approaches:

### **Acute Kidney Disease:**
- **Supportive Care:** Address the underlying cause, such as treating infections, managing medications, or correcting dehydration.
- **Medications:** Diuretics may be used to manage fluid overload, and electrolyte imbalances may be corrected.
- **Dietary Restrictions:** A low-protein, low-sodium diet may be recommended.

### **Chronic Kidney Disease (CKD):**
- **Lifestyle Changes:**
  - **Diet:** A low-sodium, low-protein, and low-potassium diet may be prescribed. Foods high in potassium (e.g., bananas, nuts) and phosphorus (e.g., dairy products) may need to be limited.
  - **Weight Management:** Maintaining a healthy weight and exercising regularly can help control blood pressure and blood sugar.
  - **Quit Smoking:** Smoking worsens kidney disease.
- **Medications:**
  - **ACE Inhibitors or ARBs:** To control high blood pressure and redu

In [19]:
res["chat_history"]

[HumanMessage(content='Tell me about the kidney disease?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='<think>\nOkay, I need to figure out how to answer the question "Tell me about the kidney disease?" using the provided context. Let me read through the context carefully.\n\nFirst, the context starts by explaining that kidney disease, also known as renal disease, reduces kidney function. It describes the kidneys as bean-shaped, fist-sized organs located below the rib cage. Their main function is filtering blood to remove waste products, which are harmful if not removed. The kidneys also regulate water and electrolyte levels, produce renin for blood pressure, and calcitriol for bone health.\n\nNext, the context discusses types of kidney disease. It mentions congenital kidney diseases present at birth, like Polycystic Kidney Disease (PKD), which is rare and causes fluid-filled cysts that replace blood-filtering units. PKD can be inherited or develop from long-term da

In [20]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='Tell me about the kidney disease?', additional_kwargs={}, response_metadata={}), AIMessage(content='<think>\nOkay, I need to figure out how to answer the question "Tell me about the kidney disease?" using the provided context. Let me read through the context carefully.\n\nFirst, the context starts by explaining that kidney disease, also known as renal disease, reduces kidney function. It describes the kidneys as bean-shaped, fist-sized organs located below the rib cage. Their main function is filtering blood to remove waste products, which are harmful if not removed. The kidneys also regulate water and electrolyte levels, produce renin for blood pressure, and calcitriol for bone health.\n\nNext, the context discusses types of kidney disease. It mentions congenital kidney diseases present at birth, like Polycystic Kidney Disease (PKD), which is rare and causes fluid-filled cysts that replace blood-filtering units. PKD